In [1]:
import pandas as pd
import io
import requests
import numpy as np
import os
import matplotlib.pyplot as plt
import pylab as pl
import tensorflow.contrib.learn as skflow
import string

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
from sklearn import preprocessing
%matplotlib inline

path = "UNSW-NB15_1.csv"
# This file is a CSV, just no CSV extension or headers
df = pd.read_csv(path, header=None)

Using TensorFlow backend.
/miniconda3/envs/tf_jupyter/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
df.dropna(inplace=True,axis=1) # For now, just drop NA's (rows with missing values)

# The CSV file has no column heads, so add them
df.columns = [
    'srcip',
    'sport',
    'dstip',
    'dsport',
    'proto',
    'state',
    'dur',
    'sbyte',
    'dbytes',
    'sttl',
    'dttl',
    'sloss',
    'dloss',
    'service',
    'Sload',
    'Dload',
    'Spkts',
    'Dpkts',
    'swin',
    'dwin',
    'stcpb',
    'dtcpb',
    'smeansz',
    'dmeansz',
    'trans',
    'res',
    'Sjit',
    'Djit',
    'Stime', #QUITAR
    'Ltime', #QUITAR
    'Sintpkt',
    'Dintpkt',
    'tcprtt',
    'synack',
    'ackdat',
    'is_sm_ips_ports',
    'ct_state_ttl',
    'ct_flw_http_mthd',
    'is_ftp_login',
    'ct_ftp_cmd',
    'ct_srv_src',
    'ct_srv_dst',
    'ct_dst_ltm',
    'ct_src_ ltm', 
    'ct_src_dport_ltm',
    'ct_dst_sport_lt', 
    'ct_dst_src_ltm',
    'attack_cat',
    'Label'
]

In [3]:
df[0:5]

,srcip,sport,dstip,dsport,proto,state,sbyte,dbytes,sttl,dttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_lt,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,...,0,0,3,7,1,3,1,1,1,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,0,2,4,2,3,1,1,2,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,0,12,8,1,2,2,1,1,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,0,6,9,1,1,1,1,1,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,0,7,9,1,1,1,1,1,0


In [4]:
df['dstip']

0          149.171.126.6
1          149.171.126.9
2          149.171.126.7
3          149.171.126.5
4          149.171.126.0
5          149.171.126.9
6          149.171.126.4
7            10.40.182.3
8          149.171.126.6
9          149.171.126.4
10           10.40.170.2
11           10.40.170.2
12           10.40.182.3
13         149.171.126.5
14         149.171.126.9
15         149.171.126.6
16         149.171.126.4
17         149.171.126.9
18         149.171.126.6
19         149.171.126.5
20        149.171.126.18
21        149.171.126.16
22        149.171.126.16
23         149.171.126.9
24         149.171.126.8
25         149.171.126.2
26         149.171.126.6
27         149.171.126.1
28         149.171.126.9
29         149.171.126.4
               ...      
699971     149.171.126.1
699972     149.171.126.1
699973     149.171.126.3
699974     149.171.126.8
699975     149.171.126.5
699976     149.171.126.7
699977     149.171.126.5
699978     149.171.126.7
699979     149.171.126.1


In [5]:
df['sport']

0          1390
1         33661
2          1464
3          3593
4         49664
5         32119
6          2142
7             0
8         40726
9         12660
10            0
11            0
12            0
13        48847
14        24266
15        10393
16        62539
17        21270
18         8989
19        49346
20        21223
21        23357
22        13284
23         4192
24        26872
25        24946
26         5685
27        22848
28        28565
29        46719
          ...  
699971    46807
699972    60323
699973    65260
699974    36648
699975    14404
699976    60849
699977    14472
699978    18827
699979    24528
699980    19461
699981    35327
699982     9969
699983    33100
699984    12691
699985    18895
699986    21661
699987    10213
699988    31786
699989    52262
699990    28517
699991    34947
699992    48114
699993    48812
699994    48114
699995     8543
699996    12520
699997    18895
699998    30103
699999    30388
700000     6055
Name: sport, Length: 700

In [6]:
df['sbyte']

0          0.001055
1          0.036133
2          0.001119
3          0.001209
4          0.001169
5          0.078339
6          0.001134
7          0.000000
8          0.001126
9          0.001167
10         0.000000
11         0.000000
12         0.000000
13         0.001093
14         0.001851
15         0.001749
16         0.001128
17         0.005153
18         0.004898
19         0.001111
20         0.000021
21         0.240139
22         2.390390
23         0.001101
24         0.001082
25         0.001122
26         0.001141
27         0.001164
28         0.001127
29         0.001073
            ...    
699971     1.241928
699972     0.006813
699973     0.053071
699974     0.001091
699975     0.178620
699976     0.001030
699977     0.390960
699978     0.001167
699979     0.000987
699980     2.074809
699981    15.268318
699982     0.099992
699983     1.140256
699984     0.310973
699985     1.402957
699986     1.010208
699987     0.074639
699988     0.001667
699989     0.006200


In [7]:
df['Sload']

0          dns
1            -
2          dns
3          dns
4          dns
5            -
6          dns
7            -
8          dns
9          dns
10           -
11           -
12           -
13         dns
14           -
15           -
16         dns
17           -
18           -
19         dns
20           -
21        http
22        http
23         dns
24         dns
25         dns
26         dns
27         dns
28         dns
29         dns
          ... 
699971       -
699972       -
699973       -
699974       -
699975       -
699976       -
699977       -
699978       -
699979       -
699980       -
699981       -
699982       -
699983       -
699984       -
699985       -
699986       -
699987       -
699988       -
699989       -
699990       -
699991       -
699992     dns
699993       -
699994       -
699995       -
699996       -
699997    http
699998       -
699999       -
700000       -
Name: Sload, Length: 700001, dtype: object

In [9]:
df['dsport']

0            53
1          1024
2            53
3            53
4            53
5           111
6            53
7             0
8            53
9            53
10            0
11            0
12            0
13           53
14        41049
15        44307
16           53
17          111
18          111
19           53
20        32780
21           80
22           80
23           53
24           53
25           53
26           53
27           53
28           53
29           53
          ...  
699971       80
699972     5190
699973      143
699974       53
699975    41875
699976       53
699977    39979
699978       53
699979       53
699980     6881
699981       80
699982     8650
699983       80
699984    14273
699985       80
699986       80
699987    46030
699988    14228
699989    51065
699990       80
699991    49439
699992       53
699993       53
699994       53
699995    45536
699996    31010
699997       80
699998     5190
699999      111
700000    54145
Name: dsport, Length: 70

In [10]:
df['proto'].dtype

dtype('O')

In [11]:
df['proto']

0         udp
1         udp
2         udp
3         udp
4         udp
5         udp
6         udp
7         arp
8         udp
9         udp
10        arp
11        arp
12        arp
13        udp
14        udp
15        udp
16        udp
17        udp
18        udp
19        udp
20        udp
21        tcp
22        tcp
23        udp
24        udp
25        udp
26        udp
27        udp
28        udp
29        udp
         ... 
699971    tcp
699972    tcp
699973    tcp
699974    udp
699975    tcp
699976    udp
699977    tcp
699978    udp
699979    udp
699980    tcp
699981    tcp
699982    tcp
699983    tcp
699984    tcp
699985    tcp
699986    tcp
699987    tcp
699988    udp
699989    tcp
699990    tcp
699991    tcp
699992    udp
699993    udp
699994    udp
699995    tcp
699996    tcp
699997    tcp
699998    tcp
699999    udp
700000    tcp
Name: proto, Length: 700001, dtype: object

In [12]:
df['state'].dtype

dtype('O')

In [13]:
df['dur'].dtype

KeyError: 'dur'

In [14]:
df['sbyte'].dtype

dtype('float64')

In [16]:
df['dbytes']


0           132
1           528
2           146
3           132
4           146
5           568
6           132
7            46
8           146
9           132
10           46
11           46
12           46
13          132
14          528
15          528
16          132
17          568
18          568
19          132
20          728
21          918
22         1362
23          132
24          132
25          132
26          146
27          146
28          132
29          132
          ...  
699971     1684
699972     1064
699973     7818
699974      146
699975     4238
699976      146
699977     5174
699978      146
699979      146
699980    13870
699981    18162
699982     4342
699983     1684
699984     3806
699985    19410
699986     1684
699987     5174
699988      536
699989     4160
699990     1684
699991      424
699992      146
699993      132
699994      146
699995     4342
699996      320
699997    19410
699998     2158
699999      568
700000     4238
Name: dbytes, Length: 70

In [22]:
df['Sload']

0          dns
1            -
2          dns
3          dns
4          dns
5            -
6          dns
7            -
8          dns
9          dns
10           -
11           -
12           -
13         dns
14           -
15           -
16         dns
17           -
18           -
19         dns
20           -
21        http
22        http
23         dns
24         dns
25         dns
26         dns
27         dns
28         dns
29         dns
          ... 
699971       -
699972       -
699973       -
699974       -
699975       -
699976       -
699977       -
699978       -
699979       -
699980       -
699981       -
699982       -
699983       -
699984       -
699985       -
699986       -
699987       -
699988       -
699989       -
699990       -
699991       -
699992     dns
699993       -
699994       -
699995       -
699996       -
699997    http
699998       -
699999       -
700000       -
Name: Sload, Length: 700001, dtype: object

In [23]:
# The CSV file has no column heads, so add them
df.columns = [
    'srcip',
    'sport',
    'dstip',
    'dsport',
    'proto',
    'state',
    'dur',
    'sbyte',
    'dbytes',
    'sttl',
    'dttl',
    'sloss',
    'dloss',
    'service',
    'Sload',
    'Dload',
    'Spkts',
    'Dpkts',
    'swin',
    'dwin',
    'stcpb',
    'dtcpb',
    'smeansz',
    'dmeansz',
    'trans',
    'res',
    'Sjit',
    'Djit',
    'Stime', #QUITAR
    'Ltime', #QUITAR
    'Sintpkt',
    'Dintpkt',
    'tcprtt',
    'synack',
    'ackdat',
    'is_sm_ips_ports',
    'ct_state_ttl',
    'ct_flw_http_mthd',
    'is_ftp_login',
    'ct_ftp_cmd',
    'ct_srv_src',
    'ct_srv_dst',
    'ct_dst_ltm',
    'ct_src_ ltm', 
    'ct_src_dport_ltm',
    'ct_dst_sport_lt', 
    'ct_dst_src_ltm',
    'attack_cat',
    'Label'
]

ValueError: Length mismatch: Expected axis has 48 elements, new values have 49 elements

In [24]:
df['attack_cat']

0         1
1         2
2         1
3         1
4         1
5         2
6         1
7         2
8         1
9         1
10        2
11        2
12        2
13        1
14        2
15        2
16        1
17        2
18        2
19        1
20        1
21        1
22        1
23        1
24        1
25        1
26        1
27        1
28        1
29        1
         ..
699971    1
699972    1
699973    1
699974    1
699975    3
699976    1
699977    3
699978    1
699979    1
699980    1
699981    1
699982    1
699983    1
699984    1
699985    2
699986    1
699987    1
699988    3
699989    3
699990    1
699991    4
699992    2
699993    1
699994    2
699995    1
699996    4
699997    2
699998    2
699999    3
700000    2
Name: attack_cat, Length: 700001, dtype: int64

In [25]:
df['Label']

0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        1
21        1
22        1
23        0
24        0
25        0
26        0
27        0
28        0
29        0
         ..
699971    0
699972    0
699973    0
699974    0
699975    0
699976    0
699977    0
699978    0
699979    0
699980    0
699981    0
699982    0
699983    0
699984    0
699985    0
699986    0
699987    0
699988    0
699989    0
699990    0
699991    0
699992    0
699993    0
699994    0
699995    0
699996    0
699997    0
699998    0
699999    0
700000    0
Name: Label, Length: 700001, dtype: int64

In [26]:
df['ct_dst_src_ltm']

0         1
1         1
2         1
3         1
4         1
5         1
6         1
7         2
8         1
9         1
10        2
11        2
12        2
13        1
14        1
15        1
16        1
17        1
18        1
19        1
20        1
21        1
22        1
23        1
24        1
25        1
26        1
27        1
28        1
29        1
         ..
699971    1
699972    1
699973    1
699974    1
699975    1
699976    1
699977    1
699978    1
699979    1
699980    1
699981    1
699982    1
699983    1
699984    1
699985    2
699986    1
699987    1
699988    1
699989    1
699990    1
699991    1
699992    2
699993    1
699994    2
699995    1
699996    1
699997    2
699998    1
699999    1
700000    1
Name: ct_dst_src_ltm, Length: 700001, dtype: int64